# WIP notebook to update interconnection FYI data

In [3]:
import pandas as pd
from dbcp.extract.helpers import cache_gcs_archive_file_locally

In [21]:
pd.set_option('display.max_columns', None)

# Raw Data

In [6]:
# Save old FYI data locally
old_fyi_path = cache_gcs_archive_file_locally("gs://dgm-archive/interconnection.fyi/interconnection_fyi_dataset_2025-09-01.csv")
# old_br_path = cache_gcs_archive_file_locally("gs://dgm-archive/ballot_ready/climate_partners_upcoming_races_with_counties_20250204.csv")
new_fyi_path = cache_gcs_archive_file_locally("gs://dgm-archive/interconnection.fyi/interconnection_fyi_dataset_2025-10-01.csv")

old_fyi = pd.read_csv(old_fyi_path)
new_fyi = pd.read_csv(new_fyi_path)

In [7]:
old_fyi

,unique_id,project_type,power_market,transmission_owner,canonical_transmission_owners,queue_id,project_name,actual_completion_date,proposed_completion_date,withdrawn_date,...,latitude,longitude,capacity_by_generation_type_breakdown,interconnection_voltage_kv,fips_codes,schedule_next_event_date,schedule_next_event_name,most_recent_study_date,most_recent_allocated_network_upgrade_cost,most_recent_study_url
0,aeso-p1756,Generation,AESO,Alberta Electric System Operator,Alberta Electric System Operator,P1756,P1756 TPG Canyon Creek PHES Storage Project,NaN,2027-05-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aeso-p1828,Generation,AESO,Alberta Electric System Operator,Alberta Electric System Operator,P1828,P1828 hep Energy Alderson MPC Solar,NaN,2026-03-05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aeso-p1885,Generation,AESO,Alberta Electric System Operator,Alberta Electric System Operator,P1885,P1885 Northern Lights WAGF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aeso-p1926,Generation,AESO,Alberta Electric System Operator,Alberta Electric System Operator,P1926,P1926 Solar Krafte Vauxhall,NaN,2025-12-15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aeso-p1927,Generation,AESO,Alberta Electric System Operator,Alberta Electric System Operator,P1927,P1927 Beargrass Solar,NaN,2026-10-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39702,wapa-ti-1001,Transmission,West,WAPA/BEPC/HCPD Integrated System,WAPA/BEPC/HCPD Integrated System,TI-1001,NaN,NaN,NaN,NaN,...,42.602926,-96.306326,NaN,161.0,19193,NaN,NaN,NaN,NaN,NaN
39703,wapa-ti-1301,Transmission,West,WAPA/BEPC/HCPD Integrated System,WAPA/BEPC/HCPD Integrated System,TI-1301,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,345.0,31089,NaN,NaN,NaN,NaN,NaN
39704,wapa-ti-1302,Transmission,West,WAPA/BEPC/HCPD Integrated System,WAPA/BEPC/HCPD Integrated System,TI-1302,NaN,NaN,NaN,NaN,...,41.819513,-103.942528,NaN,345.0,31157,NaN,NaN,NaN,NaN,NaN
39705,wapa-ti-1401,Transmission,West,WAPA/BEPC/HCPD Integrated System,WAPA/BEPC/HCPD Integrated System,TI-1401,NaN,NaN,NaN,NaN,...,46.834563,-96.948690,NaN,115.0,38017,NaN,NaN,NaN,NaN,NaN


In [9]:
from dbcp.transform.fyi_queue import _clean_all_fyi_projects
from dbcp.extract.fyi_queue import extract

In [10]:
raw_dfs = extract("gs://dgm-archive/interconnection.fyi/interconnection_fyi_dataset_2025-09-01.csv")

In [61]:
transformed_df = _clean_all_fyi_projects(raw_dfs["fyi_queue"])

/app/dbcp/transform/interconnection_queue_helpers.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queue["is_actionable"] = (
/app/dbcp/transform/interconnection_queue_helpers.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queue["is_nearly_certain"] = (


In [13]:
"fips_codes" in transformed_df

True

In [15]:
transformed_df["fips_codes"].value_counts()

06029                406
04013                364
06019                202
06073                200
06071                200
                    ... 
26003                  1
19089,19131            1
27033,27101,27127      1
46087                  1
30101                  1
Name: fips_codes, Length: 2918, dtype: int64

In [25]:
# look at rows with multiple fips codes
transformed_df[transformed_df.fips_codes == "27033,27101,27127"]

,project_type,power_market,transmission_owner,canonical_transmission_owners,queue_id,project_name,actual_completion_date_raw,proposed_completion_date_raw,withdrawn_date_raw,queue_date_raw,county_state_pairs,point_of_interconnection,raw_county_name,raw_state_name,canonical_generation_types,interconnection_service_type,capacity_mw,summer_capacity_mw,winter_capacity_mw,queue_status,current_phase_or_stage_raw,interconnection_status_raw,interconnection_status_fyi,interconnection_date_raw,developer,developer_raw,project_spv,utility,iso,cluster,general_comments,latitude,longitude,capacity_by_generation_type_breakdown,interconnection_voltage_kv,fips_codes,schedule_next_event_name,most_recent_allocated_network_upgrade_cost,most_recent_study_url,actual_completion_date,proposed_completion_date,withdrawn_date,queue_date,interconnection_date,schedule_next_event_date,most_recent_study_date,queue_year,is_actionable,is_nearly_certain
project_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
miso-j1315,Generation,MISO,NORTHERN STATES POWER COMPANY,Northern States Power Company,J1315,NaN,NaN,2022-06-30,2022-09-01,2019-04-29,"Cottonwood County, MN; Murray County, MN; Redw...",Lyon County - Cedar Mountain 345 kV Line Tap,Cottonwood County,MN,Wind,NRIS,600.0,600.0,600.0,withdrawn,Study Phase: GIA In Progress; Post GIA Status:...,Study Phase: GIA In Progress; Post GIA Status:...,IA Pending,NaN,NaN,NaN,NaN,NORTHERN STATES POWER COMPANY,miso,DPP-2019 West,NaN,NaN,NaN,NaN,345.0,"27033,27101,27127",NaN,367630069.0,https://cdn.misoenergy.org/GI-DPP-2019-West-Ph...,NaT,2022-06-30,2022-09-01,2019-04-29,NaT,NaT,2022-08-12,2019.0,<NA>,<NA>


In [24]:
transformed_df[transformed_df.fips_codes == "06029"]["county_state_pairs"]

project_id
caiso-100            KERN, CA
caiso-1000           KERN, CA
caiso-1003           KERN, CA
caiso-1004           KERN, CA
caiso-101            KERN, CA
                   ...       
ladwp-q76     Kern County, CA
ladwp-q78     Kern County, CA
ladwp-q79     Kern County, CA
ladwp-q80     Kern County, CA
ladwp-q89     Kern County, CA
Name: county_state_pairs, Length: 406, dtype: object

# Transformed Data: Data Warehouse
Let's compare the new and old Parquet files.

For this, the ETL process should be run in order to generate the warehouse tables locally.

`make all`

In [16]:
fyi_projects = pd.read_parquet("/app/data/output/private_data_warehouse/fyi_projects.parquet")

In [17]:
fyi_locations = pd.read_parquet("/app/data/output/private_data_warehouse/fyi_locations.parquet")

In [29]:
fyi_locations.head(1)

,project_id,raw_county_name,raw_state_name,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,latitude,longitude
0,aeso-p1756,Yellowhead County,AB,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [30]:
fyi_locations.county_id_fips.isnull().value_counts()

False    34989
True       656
Name: county_id_fips, dtype: int64

In [31]:
len(fyi_locations), len(fyi_locations[fyi_locations.county_id_fips.isnull()])/len(fyi_locations)

(35645, 0.018403703184177302)

In [28]:
transformed_df["fips_codes"].isnull().value_counts()

False    35009
True      1355
Name: fips_codes, dtype: int64

In [43]:
reg1 = r"^\d{5}$"
reg2 = r"^\d{5},\d{5}$"
reg3 = r"^\d{5}(?:,\d{5})*$"

In [34]:
fips_codes = transformed_df["fips_codes"].dropna()

In [44]:
fips_codes[(~fips_codes.str.contains(reg3, regex=True))]

Series([], Name: fips_codes, dtype: object)

In [45]:
fips_codes

project_id
arizona-public-service-not-assigned    04005
arizona-public-service-q007-061        04017
arizona-public-service-q044a           04027
arizona-public-service-q044b           04013
arizona-public-service-q1              35045
                                       ...  
wapa-ti-1001                           19193
wapa-ti-1301                           31089
wapa-ti-1302                           31157
wapa-ti-1401                           38017
wapa-ti-1502                           38053
Name: fips_codes, Length: 35009, dtype: object

In [55]:
five_digit_fips_regex = r"^(\d{5})"
clean_fips = fips_codes.str.extract(five_digit_fips_regex)[0]

In [64]:
clean_fips.name = "county_id_fips"

In [65]:
clean_fips

project_id
arizona-public-service-not-assigned    04005
arizona-public-service-q007-061        04017
arizona-public-service-q044a           04027
arizona-public-service-q044b           04013
arizona-public-service-q1              35045
                                       ...  
wapa-ti-1001                           19193
wapa-ti-1301                           31089
wapa-ti-1302                           31157
wapa-ti-1401                           38017
wapa-ti-1502                           38053
Name: county_id_fips, Length: 35009, dtype: object

In [58]:
clean_fips[clean_fips.str.len() > 5]

Series([], Name: 0, dtype: object)

In [67]:
clean_fips.rename("county_id_fips")

project_id
arizona-public-service-not-assigned    04005
arizona-public-service-q007-061        04017
arizona-public-service-q044a           04027
arizona-public-service-q044b           04013
arizona-public-service-q1              35045
                                       ...  
wapa-ti-1001                           19193
wapa-ti-1301                           31089
wapa-ti-1302                           31157
wapa-ti-1401                           38017
wapa-ti-1502                           38053
Name: county_id_fips, Length: 35009, dtype: object

In [73]:
geocoded_fips = fyi_locations.set_index("project_id")["county_id_fips"]

In [83]:
common_index = geocoded_fips.index.intersection(clean_fips.index)
matches = geocoded_fips.loc[common_index].reset_index(drop=True) == clean_fips.loc[common_index].reset_index(drop=True)

In [85]:
matches.value_counts()

True     34944
False       36
Name: county_id_fips, dtype: Int64

In [87]:
len(matches[~matches])

36

In [88]:
36/len(matches)

0.001028307006769688

In [89]:
len(matches)

35009